In [0]:
%sh mkdir /mnt/data

In [0]:
%scala
spark.conf.set("fs.azure.account.auth.type.streamingdemorgdl01.dfs.core.chinacloudapi.cn", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.streamingdemorgdl01.dfs.core.chinacloudapi.cn", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.streamingdemorgdl01.dfs.core.chinacloudapi.cn", "<appId>")
spark.conf.set("fs.azure.account.oauth2.client.secret.streamingdemorgdl01.dfs.core.chinacloudapi.cn", "<clientSecret>")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.streamingdemorgdl01.dfs.core.chinacloudapi.cn", "https://login.partner.microsoftonline.cn/<tenant>/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": "<appId>",
       "fs.azure.account.oauth2.client.secret": "<clientSecret>",
       "fs.azure.account.oauth2.client.endpoint": "https://login.partner.microsoftonline.cn/<tenant>/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://data@streamingdemorgdl01.dfs.core.chinacloudapi.cn/",
mount_point = "/mnt/data",
extra_configs = configs)

Out[30]: True

In [0]:
%sh ls /mnt/data/source

ls: cannot access '/mnt/data/source': No such file or directory

In [0]:
COVIDDF = spark.read.format('csv').options(
    header='true', inferschema='true').load("/mnt/data/source/*.csv")

COVIDDF.write.mode("append").format("delta").save("/mnt/data/covid")
print("Done")

Done

In [0]:
import os.path
import IPython
from pyspark.sql import SQLContext
display(dbutils.fs.ls("/mnt/data/covid"))

path,name,size
dbfs:/mnt/data/covid/_delta_log/,_delta_log/,0
dbfs:/mnt/data/covid/part-00000-8e23bdfe-1dfb-41b0-bf42-cb604f39a658-c000.snappy.parquet,part-00000-8e23bdfe-1dfb-41b0-bf42-cb604f39a658-c000.snappy.parquet,2209734
dbfs:/mnt/data/covid/part-00001-79e8225e-9674-4414-8509-6f0304632e51-c000.snappy.parquet,part-00001-79e8225e-9674-4414-8509-6f0304632e51-c000.snappy.parquet,2197114
dbfs:/mnt/data/covid/part-00002-ef448128-d5b9-48b0-b416-87d75c3983b0-c000.snappy.parquet,part-00002-ef448128-d5b9-48b0-b416-87d75c3983b0-c000.snappy.parquet,1608176
dbfs:/mnt/data/covid/part-00003-d1e13a93-8d19-42e3-b342-27c7129a0d02-c000.snappy.parquet,part-00003-d1e13a93-8d19-42e3-b342-27c7129a0d02-c000.snappy.parquet,1325226
dbfs:/mnt/data/covid/part-00004-22e82d13-3cb5-449a-9092-a6f6e3dd77d2-c000.snappy.parquet,part-00004-22e82d13-3cb5-449a-9092-a6f6e3dd77d2-c000.snappy.parquet,1218162
dbfs:/mnt/data/covid/part-00005-22913d45-d8ec-49f4-8d1f-78b22507f773-c000.snappy.parquet,part-00005-22913d45-d8ec-49f4-8d1f-78b22507f773-c000.snappy.parquet,1273996
dbfs:/mnt/data/covid/part-00006-77f88fd6-7856-4ff6-afbb-8b0b2d61be52-c000.snappy.parquet,part-00006-77f88fd6-7856-4ff6-afbb-8b0b2d61be52-c000.snappy.parquet,1341466
dbfs:/mnt/data/covid/part-00007-73b505c9-5a54-40ca-b020-8e218ad19fc2-c000.snappy.parquet,part-00007-73b505c9-5a54-40ca-b020-8e218ad19fc2-c000.snappy.parquet,914222


In [0]:
# Run each of these queries, preferably in a separate cmd cell for separate analysis
# create a temporary sql view for querying COVID information
COVIDTable = spark.read.format("delta").load('/mnt/data/covid')
COVIDTable.createOrReplaceTempView('COVIDTable')

ChinaCOVID = spark.sql("SELECT * FROM COVIDTable WHERE Country_Region = 'China (mainland)'")
ChinaCOVID.show(100)
                       
ChinaCOVID1 = spark.sql("SELECT Confirmed FROM COVIDTable WHERE Country_Region = 'China (mainland)' and AdminRegion1 = 'Liaoning' and Updated = '10/01/2020' ")
ChinaCOVID1.show()

+-------+----------+---------+---------------+------+------------+---------+---------------+--------+---------+----+----+----------------+------------+------------+
 ID| Updated|Confirmed|ConfirmedChange|Deaths|DeathsChange|Recovered|RecoveredChange|Latitude|Longitude|ISO2|ISO3| Country_Region|AdminRegion1|AdminRegion2|
+-------+----------+---------+---------------+------+------------+---------+---------------+--------+---------+----+----+----------------+------------+------------+
7159191|01/21/2020| 440| null| 0| null| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159194|01/22/2020| 571| 131| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159197|01/23/2020| 830| 259| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159199|01/24/2020| 1287| 457| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159201|01/25/2020| 1975| 688| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
5502008|01/26/2020| 2744| 769| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
5502009|01/27/2020| 4515| 1771| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
5502010|01/28/2020| 5974| 1459| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159209|01/29/2020| 7711| 1737| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159211|01/30/2020| 9692| 1981| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159213|01/31/2020| 11791| 2099| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159215|02/01/2020| 14380| 2589| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159217|02/02/2020| 17205| 2825| 0| 0| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159219|02/03/2020| 20438| 3233| 361| 361| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159221|02/04/2020| 24324| 3886| 425| 64| null| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159223|02/05/2020| 28018| 3694| 491| 66| 892| null|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159224|02/06/2020| 31161| 3143| 564| 73| 1153| 261|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159226|02/07/2020| 34546| 3385| 637| 73| 1540| 387|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159228|02/08/2020| 37198| 2652| 723| 86| 2050| 510|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159230|02/09/2020| 40171| 2973| 812| 89| 2649| 599|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159232|02/10/2020| 42638| 2467| 909| 97| 3281| 632|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159234|02/11/2020| 44653| 2015| 1017| 108| 3996| 715|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159236|02/12/2020| 46472| 1819| 1114| 97| 5062| 1066|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159238|02/13/2020| 48467| 1995| 1368| 254| 6205| 1143|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159240|02/14/2020| 49970| 1503| 1381| 13| 6601| 396|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
7159243|02/15/2020| 51091| 1121| 1524| 143| 8309| 1708|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339079|02/16/2020| 68584| 17493| 1666| 142| 9426| 1117|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339080|02/17/2020| 70635| 2051| 1772| 106| 10845| 1419|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339081|02/18/2020| 72428| 1793| 1870| 98| 12552| 1707|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339082|02/19/2020| 74280| 1852| 2006| 136| 13818| 1266|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339083|02/20/2020| 74675| 395| 2121| 115| 14376| 558|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
 339084|02/21/2020| 75569| 894| 2239| 118| 18264| 3888|36.56311| 103.7358| CN| CHN|China (mainland)| null| null|
